In [1]:
import random
from queue import PriorityQueue

from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)

In [12]:
import random

def createKSatProblem(numVars, clauseSize, numClauses):
    variables = []
    for i in range(numVars):
        variables.append(chr(i + 65))

    formula = "(("
    clauseGroup = []

    for i in range(clauseSize * numClauses):
        currentVar = random.choice(variables)
        variables.remove(currentVar)
        clauseGroup.append(currentVar)

        if i % clauseSize == clauseSize - 1:
            while len(clauseGroup) != 0:
                variables.append(clauseGroup.pop(0))

        isNegation = random.random()
        if isNegation < 0.5:
            formula += "~"

        formula += currentVar

        if i % clauseSize == clauseSize - 1 and i != (clauseSize * numClauses - 1):
            formula += ") and ("
        elif i != (clauseSize * numClauses - 1):
            formula += " or "

    formula += "))"

    return formula


In [13]:
def createRandomAssignment(varCount):
    return [random.randint(0, 1) for _ in range(varCount)]


In [14]:
def assessFitness(variableValues, clauseSize, varIndices, signList):
    score = 0
    currentClause = 0

    for i in range(len(varIndices)):
        if signList[i] == 'P':
            currentClause = currentClause or variableValues[varIndices[i]]
        else:
            currentClause = currentClause or (1 - variableValues[varIndices[i]])

        if i % clauseSize == clauseSize - 1 and currentClause == 1:
            score += 1
            currentClause = 0

    return score


In [15]:
def hillClimbSolver(varValues, maxDepth, clauseSize, varIndices, signList):
    steps = 0
    while steps < maxDepth:
        currentScore = assessFitness(varValues, clauseSize, varIndices, signList)

        if currentScore == len(varIndices):
            return varValues

        changeVar = None

        for var in varValues.keys():
            neighbor = varValues.copy()
            neighbor[var] = 1 - neighbor[var]

            neighborScore = assessFitness(neighbor, clauseSize, varIndices, signList)
            if neighborScore > currentScore:
                currentScore = neighborScore
                changeVar = var

        steps += 1
        if changeVar is not None:
            varValues[changeVar] = 1 - varValues[changeVar]

    return varValues


In [17]:
class PrioritizedState:
    def __init__(self, priority, state):
        self.priority = priority
        self.state = state

    def __lt__(self, other):
        return self.priority < other.priority

def beamSearchSolver(initialState, clauseSize, varIndices, signList, beamWidth, maxSteps):
    beam = PriorityQueue()
    currentState = initialState
    currentScore = assessFitness(currentState, clauseSize, varIndices, signList)
    beam.put(PrioritizedState(-currentScore, initialState))
    stepCount = 0

    while not beam.empty() and stepCount < maxSteps:
        state = beam.get()
        currentState = state.state
        currentScore = -state.priority

        if currentScore == len(varIndices):
            return currentState

        for var in currentState.keys():
            neighbor = currentState.copy()
            neighbor[var] = 1 - neighbor[var]

            neighborScore = assessFitness(neighbor, clauseSize, varIndices, signList)

            if beam.qsize() < beamWidth:
                beam.put(PrioritizedState(-neighborScore, neighbor))
            else:
                lowestState = beam.get()
                if neighborScore > (-lowestState.priority):
                    beam.put(PrioritizedState(-neighborScore, neighbor))
                else:
                    beam.put(lowestState)

            stepCount += 1

    return currentState

In [18]:
def flipVariableValue(currentState):
    varToFlip = random.choice(list(currentState))
    currentState[varToFlip] = 1 - currentState[varToFlip]
    return currentState

In [19]:
def swapVariableValues(currentState):
    varA = random.choice(list(currentState))
    varB = random.choice(list(currentState))

    while varB == varA:
        varB = random.choice(list(currentState))

    tempValue = currentState[varA]
    currentState[varA] = currentState[varB]
    currentState[varB] = tempValue

    return currentState

In [20]:
def flipFirstVariableValue(currentState):
    firstVar = list(currentState.keys())[0]
    currentState[firstVar] = 1 - currentState[firstVar]
    return currentState


In [21]:
def exploreVariableNeighborhood(initialState, clauseSize, varIndices, signList, maxSteps):
    stepCount = 0
    currentState = initialState

    while stepCount < maxSteps:
        currentState = initialState
        currentScore = assessFitness(initialState, clauseSize, varIndices, signList)

        if currentScore == len(varIndices):
            return currentState

        neighbor1 = flipVariableValue(currentState.copy())
        neighbor2 = swapVariableValues(currentState.copy())
        neighbor3 = flipFirstVariableValue(currentState.copy())

        score1 = assessFitness(neighbor1, clauseSize, varIndices, signList)
        score2 = assessFitness(neighbor2, clauseSize, varIndices, signList)
        score3 = assessFitness(neighbor3, clauseSize, varIndices, signList)

        bestScore = max(score1, score2, score3)
        if bestScore > currentScore:
            currentScore = bestScore
            if currentScore == score1:
                currentState = neighbor1
            elif currentScore == score2:
                currentState = neighbor2
            else:
                currentState = neighbor3

        stepCount += 1

    return currentState


In [24]:
n = 25
k = 3
m = 1000
problem = createKSatProblem(n,k,m)
numVars = set()
variables = []
posOrNeg = []

prevNeg = False

for i in range(len(problem)):
  if problem[i] == '~':
    prevNeg = True
  elif ord(problem[i]) >= 65 and ord(problem[i]) <= 90:
    if prevNeg == True:
      posOrNeg.append('N')
      prevNeg = False
    else:
      posOrNeg.append('P')

    variables.append(problem[i])
    numVars.add(problem[i])


values = createRandomAssignment(len(numVars))
startState = dict()

index = 0
for c in numVars:
  startState[c] = values[index]
  index += 1

print(startState)
print("Starting State Fitness: ", assessFitness(startState, k, variables, posOrNeg))
solution  = hillClimbSolver(startState.copy(), 100, k, variables, posOrNeg)
print("Hill Climbing Solution Fitness: ", assessFitness(solution, k, variables, posOrNeg))
solution = beamSearchSolver(startState.copy(), k, variables, posOrNeg, 3, 1000)
print("Beam Search Solution Fitness (Beam-Width = 3): ", assessFitness(solution, k, variables, posOrNeg))
solution = beamSearchSolver(startState.copy(), k, variables, posOrNeg, 4, 1000)
print("Beam Search Solution Fitness (Beam-Width = 4): ", assessFitness(solution, k, variables, posOrNeg))


print("Neighbour 1: ", flipVariableValue(startState.copy()))
print("Neighbour 2: ", swapVariableValues(startState.copy()))
print("Neighbour 3: ", flipFirstVariableValue(startState.copy()))

solution = exploreVariableNeighborhood(startState.copy(), k, variables, posOrNeg, 1000)
print("Variable-Neigbourhood-Descent Fitness: ", assessFitness(solution, k, variables, posOrNeg))


{'A': 0, 'K': 1, 'Q': 1, 'Y': 0, 'L': 1, 'P': 1, 'T': 1, 'E': 1, 'N': 1, 'O': 0, 'X': 0, 'D': 0, 'H': 1, 'S': 0, 'I': 0, 'G': 0, 'V': 1, 'F': 0, 'J': 0, 'R': 1, 'W': 1, 'B': 1, 'C': 0, 'M': 1, 'U': 0}
Starting State Fitness:  880
Hill Climbing Solution Fitness:  915
Beam Search Solution Fitness (Beam-Width = 3):  914
Beam Search Solution Fitness (Beam-Width = 4):  914
Neighbour 1:  {'A': 0, 'K': 1, 'Q': 1, 'Y': 0, 'L': 1, 'P': 1, 'T': 1, 'E': 1, 'N': 1, 'O': 0, 'X': 0, 'D': 0, 'H': 1, 'S': 0, 'I': 0, 'G': 0, 'V': 1, 'F': 1, 'J': 0, 'R': 1, 'W': 1, 'B': 1, 'C': 0, 'M': 1, 'U': 0}
Neighbour 2:  {'A': 0, 'K': 1, 'Q': 1, 'Y': 0, 'L': 1, 'P': 1, 'T': 1, 'E': 1, 'N': 1, 'O': 0, 'X': 0, 'D': 0, 'H': 1, 'S': 0, 'I': 0, 'G': 0, 'V': 1, 'F': 0, 'J': 0, 'R': 1, 'W': 1, 'B': 1, 'C': 0, 'M': 1, 'U': 0}
Neighbour 3:  {'A': 1, 'K': 1, 'Q': 1, 'Y': 0, 'L': 1, 'P': 1, 'T': 1, 'E': 1, 'N': 1, 'O': 0, 'X': 0, 'D': 0, 'H': 1, 'S': 0, 'I': 0, 'G': 0, 'V': 1, 'F': 0, 'J': 0, 'R': 1, 'W': 1, 'B': 1, 'C': 0, 